<a href="https://colab.research.google.com/github/Dheeraj024/MachineSVM/blob/main/ML_SVM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
import os,time
from sklearn.multiclass import  OneVsRestClassifier
from sklearn.metrics import accuracy_score


In [47]:
hyp_data = sio.loadmat("/content/sample_data/Indian_pines_corrected.mat")
gt_data = sio.loadmat("/content/sample_data/Indian_pines_gt.mat")

In [48]:
data = hyp_data[sorted(hyp_data.keys())[-1]]

def applyPCA(X, n_components=30, seed=1):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=n_components, whiten=True, random_state=seed)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], n_components))
    return newX

In [49]:
data.shape


(145, 145, 200)

In [50]:
data = applyPCA(data)
data.shape

labels = gt_data[sorted(gt_data.keys())[-1]]

print(data.shape)
print(labels.shape)

data = data.reshape(data.shape[0]*data.shape[1],data.shape[2])
data.shape

labels = labels.reshape(-1,1)
labels.shape

(145, 145, 30)
(145, 145)


(21025, 1)

In [51]:
unique, counts = np.unique(labels, return_counts=True)

In [52]:
print(unique,counts)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] [10776    46  1428   830   237   483   730    28   478    20   972  2455
   593   205  1265   386    93]


In [53]:
count_sort_ind = np.argsort(-counts)
print(unique[count_sort_ind])
counts[count_sort_ind]

[ 0 11  2 14 10  3  6 12  5  8 15  4 13 16  1  7  9]


array([10776,  2455,  1428,  1265,   972,   830,   730,   593,   483,
         478,   386,   237,   205,    93,    46,    28,    20])

In [54]:
data.shape

(21025, 30)

In [55]:
combine = np.concatenate((data,labels),axis=1)
combine

array([[ 0.96876849,  0.4801512 ,  0.09501285, ..., -0.08308338,
        -0.3144414 ,  3.        ],
       [ 1.08206297, -0.66688618,  0.45761543, ..., -1.70082187,
         3.01117831,  3.        ],
       [ 1.11968468, -1.01852852,  0.64112087, ...,  0.83300539,
        -0.37002131,  3.        ],
       ...,
       [-1.42368833, -0.37320899,  0.52039646, ...,  0.37675531,
         1.08857678,  0.        ],
       [-1.255093  ,  0.26308297,  0.88456889, ...,  0.01608009,
        -0.01004176,  0.        ],
       [-1.26926721,  0.11428131,  0.60833014, ...,  0.53927323,
         0.54789815,  0.        ]])

In [56]:
combine = np.delete(combine,np.where(combine[:,-1]==0),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==1),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==2),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==4),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==5),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==7),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==8),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==9),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==11),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==12),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==13),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==15),axis=0)
combine = np.delete(combine,np.where(combine[:,-1]==16),axis=0)



In [58]:
combine.shape

(3797, 31)

In [59]:
df = pd.DataFrame(combine)

In [60]:
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.968768,0.480151,0.095013,0.125597,-0.835529,-0.153065,0.340914,-0.808389,0.739768,-0.196726,...,-0.680243,-0.742375,0.309232,0.706960,-0.505994,0.346319,-0.390967,-0.083083,-0.314441,3.0
1,1.082063,-0.666886,0.457615,-0.932147,0.283993,-0.641161,0.513249,0.639667,-1.369898,0.276044,...,-0.074711,0.634126,0.264466,-0.794392,-0.303542,1.126113,-0.518024,-1.700822,3.011178,3.0
2,1.119685,-1.018529,0.641121,-1.340924,0.498481,-0.292635,0.437308,0.546881,1.651552,-0.565767,...,0.060651,0.936132,0.691308,-1.013655,-0.927224,0.821306,0.184328,0.833005,-0.370021,3.0
3,1.079131,-0.780896,0.465640,-0.886678,0.281052,-0.681895,0.634920,0.624044,-0.905912,0.162279,...,0.120929,0.673786,0.522048,-0.722938,-0.164347,1.201609,-0.673626,-1.598063,2.643712,3.0
4,0.969944,0.111926,-0.030070,-0.163259,-0.706674,-0.974477,1.271296,-0.917720,-0.543372,-0.199309,...,-0.232870,-0.182736,0.318033,0.060125,-0.217072,0.644748,-0.392546,0.256941,0.084382,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,0.432399,0.610402,-0.831501,0.205103,-0.556665,-0.939452,0.962175,-0.681345,-0.936592,-1.247825,...,0.082421,-0.322628,0.210611,0.335818,0.320615,0.588352,0.401329,0.061948,-0.113360,10.0
3793,0.685478,0.183004,-0.901307,-0.785385,-0.547188,-0.687757,1.049118,-1.134668,-0.904740,-0.039499,...,0.212918,0.183814,0.066596,-0.422357,0.315025,0.097456,-0.101163,-2.104280,2.814826,10.0
3794,0.781909,-0.067255,-0.826451,-1.684310,0.114611,-0.596822,0.765319,-0.107766,-1.028734,-0.002581,...,0.338247,0.481747,-0.406540,-1.261398,-0.126463,-0.244227,0.425079,0.476767,0.105427,10.0
3795,0.848430,-0.141022,-0.788488,-1.763317,-0.061514,-0.644363,0.712839,-0.104215,-0.610148,-0.137920,...,0.301796,0.385094,-0.050215,-1.148064,-0.531813,0.156637,0.437491,0.561004,0.136535,10.0


In [67]:
data = np.array(combine)[:,:-1]
labels = np.array(combine)[:,-1]

In [68]:
print(data.shape)
print(labels.shape)

(3797, 30)
(3797,)


In [69]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

In [70]:
C_values = [0.1, 1, 10, 100]
kernel_type = ['linear','poly','rbf']
accuracy = []

In [71]:
for kernel in kernel_type:
  for C in C_values:
      # Initialize SVM classifier with current C value
      svm = SVC(kernel=kernel, C=C)

      start_time1 = time.time()
      svm_ovr = OneVsRestClassifier(svm).fit(X_train, y_train)
      Time_taken_ovr = time.time() - start_time1
      y_test_ovr = svm_ovr.predict(X_test)

      acc_ovr = accuracy_score(y_test, y_test_ovr)
      accuracy.append(acc_ovr)

      
      # # Train SVM classifier
      # svm.fit(X_train, y_train)
      
      # # Get number of support vectors
      # n_support_vectors = svm.n_support_
      
      # # Print number of support vectors for current C value
      # print(f"C = {C}: Number of support vectors = {sum(n_support_vectors)}")

In [72]:
print(accuracy)

[0.9175438596491228, 0.9219298245614035, 0.9271929824561403, 0.9254385964912281, 0.8929824561403509, 0.9228070175438596, 0.9692982456140351, 0.968421052631579, 0.8903508771929824, 0.9517543859649122, 0.980701754385965, 0.9771929824561404]
